# SQLAlchemy and Alembic

<br>
<br>
<br>
<br>

<div style="text-align: right"> SAP Labs Korea </div>
<div style="text-align: right"> 최영선 </div>


<img align="middle" src="https://2.bp.blogspot.com/-u5RzeZjX5Ew/T7CzT9vFgyI/AAAAAAAABJ8/fDbn24qYajg/s1600/Isaac-Newton.jpg" >

<div style="text-align: center">"If I have seen future. it is by importing from the code of giants" </div>
<div style="text-align: right"> Definitely Not isaac Newton </div>

## What is the SQLAlchemy ?




### Installing SQLAlchemy

$ pip install sqlalchemy

### Version Check

In [1]:
import sqlalchemy

print( sqlalchemy.__version__)

1.1.11


### Connectiong

In [2]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///testdb.sqlite", echo=True, convert_unicode=True)

print(engine)

Engine(sqlite:///testdb.sqlite)


### Declaring a Mapping

#### Define Models

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(32)   |
| user_id       | varchar(32)   |

| Program_enrollment      ||
| :------------ | :-----------: |
| program_id | int           |
| user_id       | int           |

#### Define Models

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

In [3]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()


class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True,  autoincrement=True)
    user_name = Column(String(32), unique=True)
    user_email = Column(String(32), unique=True)
    
    def __init__(self, user_name, user_email):
        self.user_name = user_name
        self.user_email = user_email
    
    def __repr__(self):
        return "<User('%d', '%s, %s')>" % (self.user_id, self.user_name, self.user_email)
    
class Program(Base):
    __tablename__ = "programs"
    program_id = Column(Integer, primary_key=True, autoincrement=True)
    program_name = Column(String(32), unique=True)
    
    def __init__(self, program_id, program_name):
        self.program_id = program_id
        self.program_name = program_name
        
    def __repr__(self):
        return "<Program('%d', '%s')" % (self.program_id, self.program_name)
    
class ProgramEnrollment(Base):
    __tablename__ = "program_enrollment"
    program_id = Column(Integer, primary_key = True)
    user_id = Column(Integer, primary_key = True)   
    
    def __init__(self, program_id, user_id):
        self.program_id = program_id
        self.user_id = user_id
    
    def __repr__(self):
        return "<ProgramEnrollment('%s', '%s')" % (self.program_id, self.user_id)

In [4]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()


class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True,  autoincrement=True)
    user_name = Column(String(32), unique=True)
    user_email = Column(String(32), unique=True)
    
    def __init__(self, user_name, user_email):
        self.user_name = user_name
        self.user_email = user_email
    
    def __repr__(self):
        return "<User('%d', '%s, %s')>" % (self.user_id, self.user_name, self.user_email)

### Creating a Schema

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('user_id', Integer(), table=<users>, primary_key=True, nullable=False), Column('user_name', String(length=32), table=<users>), Column('user_email', String(length=32), table=<users>), schema=None)

In [6]:
Program.__table__

Table('programs', MetaData(bind=None), Column('program_id', Integer(), table=<programs>, primary_key=True, nullable=False), Column('program_name', String(length=32), table=<programs>), schema=None)

In [7]:
ProgramEnrollment.__table__

Table('program_enrollment', MetaData(bind=None), Column('program_id', Integer(), table=<program_enrollment>, primary_key=True, nullable=False), Column('user_id', Integer(), table=<program_enrollment>, primary_key=True, nullable=False), schema=None)

In [8]:
Base.metadata.create_all(bind=engine)

2017-07-29 22:12:20,263 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-29 22:12:20,265 INFO sqlalchemy.engine.base.Engine ()
2017-07-29 22:12:20,268 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-29 22:12:20,272 INFO sqlalchemy.engine.base.Engine ()
2017-07-29 22:12:20,276 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-07-29 22:12:20,278 INFO sqlalchemy.engine.base.Engine ()
2017-07-29 22:12:20,282 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	user_id INTEGER NOT NULL, 
	user_name VARCHAR(32), 
	user_email VARCHAR(32), 
	PRIMARY KEY (user_id), 
	UNIQUE (user_name), 
	UNIQUE (user_email)
)


2017-07-29 22:12:20,283 INFO sqlalchemy.engine.base.Engine ()
2017-07-29 22:12:20,297 INFO sqlalchemy.engine.base.Engine COMMIT


### Creating a Session

In [9]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

print(session)

In [10]:
session.close()

### Adding and Updating Object

#### Adding Objects

In [11]:
Session = sessionmaker(bind=engine)

session = Session()

user = User("최영선", "yeongseon.choe@pycon.kr")

session.add(user)
session.commit()

2017-07-29 22:12:20,357 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-07-29 22:12:20,361 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-07-29 22:12:20,363 INFO sqlalchemy.engine.base.Engine ('최영선', 'yeongseon.choe@pycon.kr')
2017-07-29 22:12:20,369 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
session.add_all([
    User("배준현", "junhyun.bae@pycon.kr"),
    User("김준기", "junki.kim@pycon.kr")]
)
session.commit()

2017-07-29 22:12:20,400 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-07-29 22:12:20,402 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-07-29 22:12:20,410 INFO sqlalchemy.engine.base.Engine ('배준현', 'junhyun.bae@pycon.kr')
2017-07-29 22:12:20,420 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-07-29 22:12:20,422 INFO sqlalchemy.engine.base.Engine ('김준기', 'junki.kim@pycon.kr')
2017-07-29 22:12:20,424 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
user1 = User("강대성", "daesung.kang@pycon.kr")
user2 = User("한성준", "sungjun.han@pycon.kr")

session.bulk_save_objects([user1, user2])
session.commit()

2017-07-29 22:12:20,459 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-07-29 22:12:20,461 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-07-29 22:12:20,466 INFO sqlalchemy.engine.base.Engine (('강대성', 'daesung.kang@pycon.kr'), ('한성준', 'sungjun.han@pycon.kr'))
2017-07-29 22:12:20,473 INFO sqlalchemy.engine.base.Engine COMMIT


#### Updating Objects

In [14]:
user = session.query(User).filter_by().first() 
print(user)

2017-07-29 22:12:20,501 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-07-29 22:12:20,507 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
 LIMIT ? OFFSET ?
2017-07-29 22:12:20,508 INFO sqlalchemy.engine.base.Engine (1, 0)
<User('1', '최영선, yeongseon.choe@pycon.kr')>


In [15]:
user.user_name = 'Yeongseon Choe'
print(user)
session.commit()

<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>
2017-07-29 22:12:20,527 INFO sqlalchemy.engine.base.Engine UPDATE users SET user_name=? WHERE users.user_id = ?
2017-07-29 22:12:20,531 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe', 1)
2017-07-29 22:12:20,539 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
user = session.query(User).filter_by().first() 
print(user)

2017-07-29 22:12:20,569 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-07-29 22:12:20,573 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
 LIMIT ? OFFSET ?
2017-07-29 22:12:20,575 INFO sqlalchemy.engine.base.Engine (1, 0)
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>


#### Querying

In [17]:
for user in session.query(User):
    print(user)

2017-07-29 22:12:20,590 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
2017-07-29 22:12:20,594 INFO sqlalchemy.engine.base.Engine ()
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('3', '김준기, junki.kim@pycon.kr')>
<User('4', '강대성, daesung.kang@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>


In [18]:
for user in session.query(User).order_by(User.user_email):
    print(user)

2017-07-29 22:12:20,609 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users ORDER BY users.user_email
2017-07-29 22:12:20,612 INFO sqlalchemy.engine.base.Engine ()
<User('4', '강대성, daesung.kang@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('3', '김준기, junki.kim@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>


descending order

In [19]:
from sqlalchemy import desc
for user in session.query(User).order_by(desc(User.user_email)):
    print(user)

2017-07-29 22:12:20,627 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users ORDER BY users.user_email DESC
2017-07-29 22:12:20,630 INFO sqlalchemy.engine.base.Engine ()
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>
<User('3', '김준기, junki.kim@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('4', '강대성, daesung.kang@pycon.kr')>


- fileter method
    - equals <br>
    query.filetr(User.name == 'Yeongseon Choe')
    - not equals <br>
    query.filetr(User.name != 'Yeongseon Choe')
    - like <br>
    query.filetr(User.name.like('Yeongseon Choe')

In [20]:
session.query(User).filter(User.user_name =='Yeongseon Choe')

- Returning Lists and Scalars
    - all()
    - first()
    - one()

In [21]:
users = session.query(User).filter(User.user_name == 'Yeongseon Choe').all()
print(users)

2017-07-29 22:12:20,658 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
2017-07-29 22:12:20,660 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe',)
[<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>]


In [22]:
users = session.query(User).filter(User.user_name == 'Yeongseon Choe').first()
print(users)

2017-07-29 22:12:20,674 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2017-07-29 22:12:20,677 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe', 1, 0)
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>


In [23]:
users = session.query(User).filter(User.user_name == 'Yeongseon Choe').one()
print(users)

2017-07-29 22:12:20,690 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
2017-07-29 22:12:20,692 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe',)
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>


#### Deleting Object

In [27]:
user = session.query(User).filter(User.user_name == 'Yeongseon Choe').first()
session.delete(user)
session.commit()

2017-07-29 22:13:13,937 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2017-07-29 22:13:13,939 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe', 1, 0)
2017-07-29 22:13:13,942 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.user_id = ?
2017-07-29 22:13:13,943 INFO sqlalchemy.engine.base.Engine (1,)
2017-07-29 22:13:13,948 INFO sqlalchemy.engine.base.Engine COMMIT


#### Define Models

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(32)   |
| user_id       | varchar(32)   |

| Program_enrollment      ||
| :------------ | :-----------: |
| program_id | int           |
| user_id       | int           |

In [25]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()
    
class Program(Base):
    __tablename__ = "programs"
    program_id = Column(Integer, primary_key=True, autoincrement=True)
    program_name = Column(String(32), unique=True)
    
    def __init__(self, program_id, program_name):
        self.program_id = program_id
        self.program_name = program_name
        
    def __repr__(self):
        return "<Program('%d', '%s')" % (self.program_id, self.program_name)
    
class ProgramEnrollment(Base):
    __tablename__ = "program_enrollment"
    program_id = Column(Integer, primary_key = True)
    user_id = Column(Integer, primary_key = True)   
    
    def __init__(self, program_id, user_id):
        self.program_id = program_id
        self.user_id = user_id
    
    def __repr__(self):
        return "<ProgramEnrollment('%s', '%s')" % (self.program_id, self.user_id)
    
# Base.metadata.create_all(bind=engine)

## Alembic

### Database migration tool
- creation
- management
- invocation of change managment

### Installing Alembic

$ pip install alembic

### Usage

usage: alembic [-h] [-c CONFIG] [-n NAME] [-x X] [--raiseerr]
               {branches,current,downgrade,edit,heads,history,init,list_templates,merge,revision,show,stamp,upgrade}     


### Creating an Environment

$ alembic init 'alembic' <br>

```

project
│          alembic.ini
└─── alembic
      │    env.py
      │    README
      │    script.py.mako      
      └─ versions
```


  
$ vi alembic.ini

<pre>
# A generic, single database configuration.

[alembic]
# path to migration scripts
script_location = alembic

# template used to generate migration files
# file_template = %%(rev)s_%%(slug)s

# timezone to use when rendering the date
# within the migration file as well as the filename.
# string value is passed to dateutil.tz.gettz()
# leave blank for localtime
# timezone =

# max length of characters to apply to the
# "slug" field
#truncate_slug_length = 40

# set to 'true' to run the environment during
# the 'revision' command, regardless of autogenerate
# revision_environment = false

# set to 'true' to allow .pyc and .pyo files without
# a source .py file to be detected as revisions in the
# versions/ directory
# sourceless = false

# version location specification; this defaults
# to alembic/versions.  When using multiple version
# directories, initial revisions must be specified with --version-path
# version_locations = %(here)s/bar %(here)s/bat alembic/versions

# the output encoding used when revision files
# are written from script.py.mako
# output_encoding = utf-8

sqlalchemy.url = sqlite:///testdb.sqlite


# Logging configuration
[loggers]
keys = root,sqlalchemy,alembic

[handlers]
keys = console

[formatters]
keys = generic

[logger_root]
level = WARN
handlers = console
qualname =

[logger_sqlalchemy]
level = WARN
handlers =
qualname = sqlalchemy.engine

[logger_alembic]
level = INFO
handlers =
qualname = alembic

[handler_console]
class = StreamHandler
args = (sys.stderr,)
level = NOTSET
formatter = generic

[formatter_generic]
format = %(levelname)-5.5s [%(name)s] %(message)s
datefmt = %H:%M:%S

</pre>

### Creating a Migration Script

$ alembic revision -m "create tables"

<br>
```
project
│          alembic.ini
└─── alembic
      │    env.py
      │    README
      │    script.py.mako      
      └─ versions
          │   8de6f753dcac_create_tables.pyy
```

$ vi alembic/versions/8de6f753dcac_create_tables.py

In [28]:
"""create tables

Revision ID: 8de6f753dcac
Revises: 
Create Date: 2017-07-29 22:19:30.546010

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = '8de6f753dcac'
down_revision = None
branch_labels = None
depends_on = None


def upgrade():
    pass


def downgrade():
    pass


In [ ]:
"""create tables

Revision ID: 8de6f753dcac
Revises: 
Create Date: 2017-07-29 22:19:30.546010

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = '8de6f753dcac'
down_revision = None
branch_labels = None
depends_on = None


def upgrade():
    op.create_table(
        'programs',
        sa.Column('program_id', sa.Integer, primary_key=True, autoincrement=True),
        sa.Column('program_name', sa.String(32), unique=True, nullable=False),
    )
    op.create_table(
        'program_enrollment',
        sa.Column('program_id', sa.Integer, primary_key=True),
        sa.Column('user_id', sa.Integer, primary_key=True),
    )

def downgrade():
    op.drop_table('programs')
    op.drop_table('program_enrollment')


 $ alembic.exe upgrade head
 
INFO  [alembic.runtime.migration] Context impl SQLiteImpl. <br>
INFO  [alembic.runtime.migration] Will assume non-transactional DDL. <br>
INFO  [alembic.runtime.migration] Running upgrade  -> 8de6f753dcac, create tables
